# Semantic Segmentation Algorithm
1. [Project Highlight](#Project-Highlight)
2. [Introduction](#Introduction)
3. [AWS Authentication](#AWS-Authentication)
4. [Data Preparation](#Data-Preparation)
  1. [Download data](#Download-data)
  2. [Setup Data](#Setup-data)
  3. [Upload to S3](#Upload-to-S3)
5. [Training](#Training)
6. [Hosting](#Hosting)
7. [Delete The Endpoint](#Delete-the-Endpoint)

## Project Highlight

In an effort to prototype our startup studio project, we used the camera attached to our small RC car model which is hosted on a Raspberry Pi. The camera gives us a better understanding of the obstaclese surrounding the vehicle and is streamed to AWS using Amazon Kinesis Video Streams. Once input from the stream is received we are able to then use AWS sagemarker to analyze the stream and send the output to a S3 bucket. From there a AWS Lambda function reads from the S3 bucket and kickstarts the decoding process.


## Introduction

Semantic Segmentation is the task of classifying every pixel in an image with a class from a known set of labels. The output is an integer matrix of the same shape as the input frame. Scene parsing provides complete understanding of the scene. Within the context of this project, this is critical. The mask offers us a highly condensed version of the frame with the same semantic information still encoded.

The purpose of this notebook is to create the section of our architectural design: a Pyramid Scene Parsing Network that converts the given frame to a segmented mask. More information about the Pyramid Scene Parsing Network can be found within this paper([PSP](https://arxiv.org/abs/1612.01105)). Additionally, the parameters used are the same as those mentioned in the paper previously cited.


## AWS Authentication
Authenticate the use of AWS services for the startup studio IAM role we have set up for this project. Permissions: access to SageMaker, access to a advanced computing machine, and access to all S3 buckets associated with the account.

In [ ]:
%%time
import sagemaker
from sagemaker import get_execution_role
 
role = get_execution_role()
print(role)
sess = sagemaker.Session()

Creates a S3 bucket storing training data and all the artifacts created by the model.

In [ ]:
bucket = sess.default_bucket()  
prefix = 'xporter-segmentation'
print(bucket)

Lastly, we need the Amazon SageMaker Semantic Segmentaion docker image, which is static and need not be changed.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
training_image = get_image_uri(sess.boto_region_name, 'semantic-segmentation', repo_version="latest")
print (training_image)

## Data Preparation
[Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/) is a popular computer vision dataset which is used for annual semantic segmentation challenges from 2005 to 2012. The dataset has 1464 training and 1449 validation images with 21 classes. Examples of the segmentation dataset can be seen in the [Pascal VOC Dataset page](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/segexamples/index.html). The classes are as follows:

| Label Id |     Class     |
|:--------:|:-------------:|
|     0    |   Background  |
|     1    |   Aeroplane   |
|     2    |    Bicycle    |
|     3    |      Bird     |
|     4    |      Boat     |
|    5     |     Bottle    |
|     6    |      Bus      |
|     7    |      Car      |
|     8    |      Cat      |
|     9    |     Chair     |
|    10    |      Cow      |
|    11    |  Dining Table |
|    12    |      Dog      |
|    13    |     Horse     |
|    14    |   Motorbike   |
|    15    |     Person    |
|    16    |  Potted Plant |
|    17    |     Sheep     |
|    18    |      Sofa     |
|    19    |     Train     |
|    20    |  TV / Monitor |
|    255   | Hole / Ignore |

In this notebook, we will use the data sets from 2012. While using the Pascal VOC dataset, please be aware of the  usage rights:
"The VOC data includes images obtained from the "flickr" website. Use of these images must respect the corresponding terms of use: 
* "flickr" terms of use (https://www.flickr.com/help/terms)"

### Download data
Download the Pascal VOC datasets from VOC 2012. This section only needs to be run once, after the first time we just grab everything from the S3 bucket we made. 

In [ ]:
%%time

# Download the dataset
!wget -P /tmp http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar    
# # Extract the data.
!tar -xf /tmp/VOCtrainval_11-May-2012.tar && rm /tmp/VOCtrainval_11-May-2012.tar

### Setup data
Move the downloaded data into S3 buckets with the correct directories so that it is easy to sort. This pattern is the same as the recommended structure from PASCAL. 

In [ ]:
import os
import shutil

# Create directory structure mimicing the s3 bucket where data is to be dumped.
VOC2012 = 'VOCdevkit/VOC2012'
os.makedirs('train', exist_ok=True)
os.makedirs('validation', exist_ok=True)
os.makedirs('train_annotation', exist_ok=True)
os.makedirs('validation_annotation', exist_ok=True)

# Create a list of all training images.
filename = VOC2012+'/ImageSets/Segmentation/train.txt'
with open(filename) as f:
    train_list = f.read().splitlines() 

# Create a list of all validation images.
filename = VOC2012+'/ImageSets/Segmentation/val.txt'
with open(filename) as f:
    val_list = f.read().splitlines() 

# Move the jpg images in training list to train directory and png images to train_annotation directory.
for i in train_list:
    shutil.copy2(VOC2012+'/JPEGImages/'+i+'.jpg', 'train/')
    shutil.copy2(VOC2012+'/SegmentationClass/'+i+'.png','train_annotation/' )

# Move the jpg images in validation list to validation directory and png images to validation_annotation directory.
for i in val_list:
    shutil.copy2(VOC2012+'/JPEGImages/'+i+'.jpg', 'validation/')
    shutil.copy2(VOC2012+'/SegmentationClass/'+i+'.png','validation_annotation/' )

The current structure of the S3 bucket should look like this:

```bash
root 
|-train/
|-train_annotation/
|-validation/
|-validation_annotation/

```

We will let AWS know that we want the image names (which are integers) in "annotation" directories to be read as labels directly.

In [ ]:
import json
label_map = { "scale": 1 }
with open('train_label_map.json', 'w') as lm_fname:
    json.dump(label_map, lm_fname)

In [ ]:
# Create channel names for the s3 bucket.
train_channel = prefix + '/train'
validation_channel = prefix + '/validation'
train_annotation_channel = prefix + '/train_annotation'
validation_annotation_channel = prefix + '/validation_annotation'
# label_map_channel = prefix + '/label_map'

### Upload to S3
Let's upload the data with the label map.

In [ ]:
%%time
# upload the appropraite directory up to s3 respectively for all directories.
sess.upload_data(path='train', bucket=bucket, key_prefix=train_channel)
sess.upload_data(path='validation', bucket=bucket, key_prefix=validation_channel)
sess.upload_data(path='train_annotation', bucket=bucket, key_prefix=train_annotation_channel)
sess.upload_data(path='validation_annotation', bucket=bucket, key_prefix=validation_annotation_channel)
# sess.upload_data(path='train_label_map.json', bucket=bucket, key_prefix=label_map_channel)

Next we need to setup an output location at S3, where the model artifact will be dumped. These artifacts are also the output of the algorithm's traning job. Let us use another channel in the same S3 bucket for this purpose.

In [ ]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

## Training
With the training data all complete, it is time to actually train the model. We will be using a advanced computer (ml.p2.xlarge) to speed up the training time. 

In [ ]:
# Create the sagemaker estimator object.
ss_model = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count = 1, 
                                         train_instance_type = 'ml.p2.xlarge',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         output_path = s3_output_location,
                                         base_job_name = 'xporter-segmentation',
                                         sagemaker_session = sess)

According to the paper the following the PSPNet(50) got strong results. The mean IoU was 41.68% and the ovrall pixel accuracy was 80.04%. As noted in the paper preformance grows with deeper networks, but we are going with just resnet 50 because speed is critical. The faster we make the segmentation process, the faster the remote driver is going to be able to make a decision about how to operate the vehicle.

In [ ]:
# Setup hyperparameters 
ss_model.set_hyperparameters(backbone='resnet-50', 
                             algorithm='psp', # PSPNet(50)                              
                             use_pretrained_model='True', # Use the pre-trained model.
                             crop_size=240,                              
                             num_classes=21, # Pascal has 21 classes, see the data prep for the list of classes.
                             epochs=10,
                             learning_rate=0.0001,                             
                             optimizer='rmsprop', 
                             lr_scheduler='poly',                          
                             mini_batch_size=16, 
                             validation_mini_batch_size=16,
                             early_stopping=True, 
                             early_stopping_patience=2, # Tolerate these many epochs if the mIoU doens't increase.
                             early_stopping_min_epochs=10, # No matter what, run these many number of epochs.                             
                             num_training_samples=num_training_samples)

Now that the hyperparameters are setup, let us prepare the handshake between our data channels and the algorithm. To do this, we need to create the `sagemaker.session.s3_input` objects from our data channels. These objects are then put in a simple dictionary, which the algorithm uses to train.

In [ ]:
distribution = 'FullyReplicated'
# Create sagemaker s3_input objects
train_data = sagemaker.session.s3_input('s3://{}/{}'.format(bucket, train_channel), 
                                        distribution=distribution, 
                                        content_type='image/jpeg', 
                                        s3_data_type='S3Prefix')

validation_data = sagemaker.session.s3_input('s3://{}/{}'.format(bucket, validation_channel),
                                             distribution=distribution, 
                                             content_type='image/jpeg', 
                                             s3_data_type='S3Prefix')

train_annotation = sagemaker.session.s3_input('s3://{}/{}'.format(bucket, train_annotation_channel), 
                                              distribution=distribution, 
                                              content_type='image/png', 
                                              s3_data_type='S3Prefix')

validation_annotation = sagemaker.session.s3_input('s3://{}/{}'.format(bucket, validation_annotation_channel), 
                                              distribution=distribution, 
                                              content_type='image/png', 
                                              s3_data_type='S3Prefix')
data_channels = {'train': train_data, 
                 'validation': validation_data,
                 'train_annotation': train_annotation, 
                 'validation_annotation':validation_annotation}

Here we want to train the model and store the model in our S3 bucket This is important because it makes it possible for this model to be used in future situations. The future situations we have in mind are when each frame comes in via the Kinesis stream.

In [ ]:
ss_model.fit(inputs=data_channels, logs=True)

# Hosting
Once the training is done, we can deploy the trained model as an Amazon SageMaker hosted endpoint. This will allow us to make predictions (or inference) from the model. Note that we don't have to host on the same instance (or type of instance) that we used to train. 

Once the endpoint is up we can test individual images by writing the following:

```

ss_predictor.content_type = 'image/jpeg'
ss_predictor.accept = 'image/png'
return_img = ss_predictor.predict(img)
```

In [ ]:
ss_predictor = ss_model.deploy(initial_instance_count=1, instance_type='ml.t2.xlarge')

## Delete the Endpoint
Noting is free in the world. So turn the endpoint off.

In [ ]:
sagemaker.Session().delete_endpoint(ss_predictor.endpoint)